In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
start_day = 0
start_month = 0
end_day = 0
end_month = 0
location = ""
locations = ['Aguayo', 'Ajax', 'Amazon', 'Anacacho', 'Appaloosa Run', 'Aquilla Lake 2', 'Aquilla Lake 1', 'Armstrong',
             'Azure Sky 1', 'Azure Sky 2', 'Baffin', 'Baird North', 'Barranca 2', 'Barranca 1', 'Barrow Ranch',
             'Barton Chapel', 'Bearkat', 'Bethel', 'Big Sampson', 'Big Spring', 'Black Jack Creek', 'Blanco Canyon',
             'Blue Summit 2', 'Blue Summit Repower', 'Blue Summit 1', 'Blue Summit 3', 'Bobcate Bluff Repower',
             'Bobcat Bluff', 'Brazos', 'Briscoe', 'Broadview Energy', 'Bruennings Breeze', 'Buckthorn',
             'Buenos Aires 1', 'Buenos Aires 2', 'Bufallo Gap 4', 'Buffalo Gap', 'Cabezon', 'Cactus Flats',
             'Callahan Divide', 'Cameron', 'Camp Springs 2', 'Canadian Breaks', 'Canyon Wind', 'Capricorn 3',
             'Capricorn 4', 'Capricorn Ridge', 'Cattleman Wind A', 'Cattleman Wind B']
def update_start_day(limit):
    
    start_day = limit
    
    #print("Start Day: "+str(start_day))

def update_end_day(limit):
    
    end_day = limit
    
    #print("End Day: "+str(end_day))

def update_start_month(limit):
    
    start_month = limit
    
    #print("Start Day: "+str(start_month))

def update_end_month(limit):
    
    end_month = limit
    
    #print("Start Day: "+str(end_month))

def update_location(limit):
    
    location = limit
    
    #print("Location: "+str(location))


In [4]:
style = {'description_width': 'initial'}
days = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
months = [1,2,3,4,5,6,7,8,9,10,11,12]

location = widgets.Dropdown(
    options=locations,
    value='Aguayo',
    description='Wind Locations',
    disabled=False,
    style=style
)

start_m = widgets.Dropdown(
    options=months,
    value=6,
    description='Start Month',
    disabled=False,
    style=style
)

start_d = widgets.Dropdown(
    options=days,
    value=15,
    description='Start Day',
    disabled=False,
    style=style
)


end_m = widgets.Dropdown(
    options=months,
    value=6,
    description='End Month',
    disabled=False,
    style=style
)

end_d = widgets.Dropdown(
    options=days,
    value=16,
    description='End Day',
    disabled=False,
    style=style
)

In [5]:
import glob
import h5py
import matplotlib.pyplot as plt
import datetime
path = "Data/Wind/"
files = glob.glob(path + "*.h5")
files.sort()
#print(files)
#s_path = "Data/Solar/"
#s_files = glob.glob(s_path + "*.h5")
#s_files.sort()
#print(s_files)



def update_graph(start_day,start_month,end_day,end_month,location):
    for i,loc in enumerate(locations):
        if loc == location:
            index = i
            break
    with h5py.File(files[index], 'r') as f:
        print(f)
        actuals = pd.DataFrame(f['actuals'][...])
        actual_time_steps = pd.to_datetime(f['time_index'][...].astype(str), errors='coerce', format='%Y-%m-%d %H:%M:%S')
    
    with h5py.File(files[index+264], 'r') as f:
        #print(f)
        forecasts= pd.DataFrame(f['forecasts'][...])
        time_intervals = pd.DataFrame(f['forecast_time'][...]) 
        time_intervals[0] = time_intervals[0].str.decode("utf-8") #Convert from byte array to string
        fcst_time_steps = pd.to_datetime(f['forecast_time'][...].astype(str), format='%Y-%m-%d %H:%M:%S%z', errors='raise', utc = True)
        percentile50 = pd.DataFrame(forecasts[49]); #50th PERCENTILE
        percentile25 = pd.DataFrame(forecasts[24]); #50th PERCENTILE
        percentile75 = pd.DataFrame(forecasts[74]); #50th PERCENTILE
        percentile5 = pd.DataFrame(forecasts[4]); #50th PERCENTILE
        percentile95 = pd.DataFrame(forecasts[94]); #50th PERCENTILE

    plt.plot(actual_time_steps, actuals[0], label = "Actuals", linewidth = 3.5)
    plt.plot(fcst_time_steps, percentile50, "r-", label= "50th Percentile")
    plt.plot(fcst_time_steps, percentile25, "g-", label= "25th Percentile")
    plt.plot(fcst_time_steps, percentile75, color = "orange", label= "75th Percentile")
    plt.plot(fcst_time_steps, percentile95, color = "purple", label= "95th Percentile")
    plt.plot(fcst_time_steps, percentile5, color = "pink", label= "5th Percentile")
    plt.xlim(datetime.datetime(2018,int(start_month),int(start_day)), datetime.datetime(2018,int(end_month),int(end_day))) # Format:(year, month, day, hour, min, sec)
    plt.xticks(rotation='vertical')
    plt.margins(0.2)
    plt.xlabel("Date/Hour")
    plt.ylabel("Power [MW]")
    plt.title("Wind Actual vs Day-Ahead Forecast")
    plt.legend()
    plt.rcParams["figure.figsize"] = (12,10)
    plt.show()

In [6]:
#import h5py
#import pandas as pd
# import s3fs
#import matplotlib.pyplot as plt
#import datetime
import math
import sklearn
from sklearn.metrics import mean_squared_error
from pathlib import Path
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import autocorrelation_plot

In [7]:
start_day1 = 0
end_day1 = 0
length = 0
location1 = ""
locations2 = ['Data/Load/Zone_Coast_load_actuals_2018.h5', 'Data/Load/Zone_East_load_actuals_2018.h5', 'Data/Load/Zone_Far_West_load_actuals_2018.h5',
             'Data/Load/Zone_West_load_actuals_2018.h5', 'Data/Load/Zone_North_load_actuals_2018.h5','Data/Load/Zone_North_Central_load_actuals_2018.h5',
             'Data/Load/Zone_Southern_load_actuals_2018.h5', 'Data/Load/Zone_South_Central_load_actuals_2018.h5']
locations1 = ['Coast Zone', 'East Zone', 'Far West Zone','West Zone', 'North Zone', 'North Central Zone', 'Southern Zone', 'South Central Zone']
def update_start_day1(limit):
    
    start_day1 = limit
    
    #print("Start Day: "+str(start_day))


def update_location1(limit):
    
    location1 = limit
    
    #print("Location: "+str(location))

def update_length(limit):
    length = limit

In [8]:
days1 = list(range(1,366))

location1 = widgets.Dropdown(
    options=locations1,
    value='Coast Zone',
    description='Load Locations',
    disabled=False,
    style=style
)


start_d1 = widgets.Dropdown(
    options=days1,
    value=15,
    description='Start Day',
    disabled=False,
    style=style
)


length1 = widgets.Dropdown(
    options=days1,
    value=12,
    description='Number of 5 min intervals to predict',
    disabled=False,
    style=style
)

In [9]:
path1 = "Data/Load/"
files1 = glob.glob(path1 + "*.h5")
files1.sort()

def update_graph1(start_day1,location1, length):
    index1 = 0
    figure, axis = plt.subplots(1, 1)
    for i,loc in enumerate(locations1):
        if loc == location:
            index1 = i
            break
    with h5py.File(files1[index1], 'r') as f:
        print(f)
        actuals = pd.DataFrame(f['actuals'][...])
        # actual_time_steps = pd.to_datetime(f['time_index'][...].astype(str), errors='coerce', format='%Y-%m-%d %H:%M:%S')
        actuals['date'] = pd.to_datetime(f['time_index'][...].astype(str), errors='coerce', format='%Y-%m-%d %H:%M:%S')
        # dates = pd.DataFrame(f['time_index'][...].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
        # df = pd.concat([actuals,actual_time_steps],axis = 1)
        # autocorrelation_plot(actuals)
        actuals.columns = ['Load','Date']
        actuals = actuals.iloc[:,[1,0]]
        # print(actuals.head())
        # plt.plot(actuals['Date'], actuals['Load'])
        # plt.show()
        #autocorrelation_plot(actuals['Load'])
    
    daysinintervals = start_day1*288
    train = actuals['Load'][(daysinintervals - 2016):daysinintervals] 
    test = actuals['Load'][daysinintervals:daysinintervals+length]
    # print(test[52560], train)
    history = [x for x in train]
    expected = list()
    predictions = list()
    dates = list()
    
    for t in range(daysinintervals,daysinintervals+length):
        model = ARIMA(history, order=(2,1,2))
        # model = ARIMA(history, order=(1,1,1))
        model_fit = model.fit()
        output = model_fit.forecast()
        # print(output)
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        date = actuals['Date'][t]
        dates.append(date)
        date = date.strftime("%Y-%m-%d %H:%M:%S")
        history.append(test[t])
        expected.append(test[t])
        #print('%d Date = %s, Predicted = %f, Expected = %f' % ((t-daysinintervals),date, yhat, obs))
    #evaluate forecasts
    output = pd.DataFrame([dates, expected, predictions])
    with open('ModelOutput.csv', 'w') as csv_file:
        output.to_csv(csv_file, index = False)
        
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(expected, predictions))
    print('Test RMSE: %.3f' % rmse)
    #plot forecasts against actual outcomes
    plt.plot(dates,expected, label = "actuals")
    plt.plot(dates,predictions, color = 'red', label = "forecast")
    plt.xticks(rotation='vertical')
    plt.margins(0.2)
    plt.xlabel("Date/Hour")
    plt.ylabel("Load [MWh]")
    plt.title("Actual vs ARIMA Forecast")
    plt.legend()
    plt.show()
    
    #autocorrelation_plot(actuals['Load'])
    
    #plot_pacf(train);
    #plot_acf(train);
    ad_fuller_result = adfuller(train)
    #print(f'ADF Statistic: {ad_fuller_result[0]}')
    #print(f'p-value: {ad_fuller_result[1]}')
    
    
    
    

In [10]:
# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

# Notes on allZonesErcot.csv
# 0 - zone east
# 1 - zone west
# 2 - zone coast
 
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        df = df.iloc[1: , :]  # skip 1st row
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
 
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X
 
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

def grade(yhat):
    aScore = 0.063
    res = ''
    if yhat < aScore:
        res = "A"
    elif yhat < 2 * aScore:
        res = "B"
    elif yhat < 3 * aScore:
        res = "C"
    else:
        res = "D"
    print("Demand risk: ", res)
        
 
# prepare the data
#path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
print('Machine learning model learning. This may take up to 90 seconds.')
path = "allZonesErcot.csv"
train_dl, test_dl = prepare_data(path)
#print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(3)
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a prediction (expect class=1)


Machine learning model learning. This may take up to 90 seconds.
MSE: 0.003, RMSE: 0.055


In [14]:
start_day2 = 0

def update_start_day2(limit):
    
    start_day2 = limit


start_d2 = widgets.Dropdown(
    options=days1,
    value=15,
    description='Start Day',
    disabled=False,
    style=style
)


In [19]:
def update_graph2(start_day2):
    rows = [[],[],[] ]
    for h in range(3):  #zone code
        for i in range(24):     # hour
            rows[h].append([h, start_day2, i ] ) 
    #output
    res = [0,0,0]
    variations = [[],[],[]]
    for h in range(3):  #zone code
        for i in range(24):
            yhat = predict(rows[h][i], model)
            res[h] += yhat
            variations[h].append(yhat[0])


    #plot
    x_axis = [i for i in range(24)]
    y_axis_west = variations[0]
    y_axis_east = variations[1]
    y_axis_coast = variations[2]

    plt.plot(x_axis, y_axis_west, label = 'Zone West', color = 'red')
    plt.plot(x_axis, y_axis_east, label = 'Zone East', color = 'green')
    plt.plot(x_axis, y_axis_coast, label = 'Zone Coast', color = 'blue')

    plt.title('Day-Ahead-Market: Load variance on '+ str(1+start_day2//30) + "/" + str(start_day2%31) + "/2022")
    plt.xlabel('Hour')
    plt.ylabel('Variation')
    res[0] = res[0]/24
    res[1] = res[1]/24
    res[2] = res[2]/24

    print('\nZone-West (red)')
    print('Predicted Demand Variation: %.5f' % res[0])
    grade(res[0])
    print('\nZone-East (green)')
    print('Predicted Demand Variation: %.5f' % res[1])
    grade(res[1])
    print('\nZone-Coast (blue)')
    print('Predicted Demand Variation: %.5f' % res[2])
    grade(res[2])

    ymin = min( min(variations[0]), min(variations[1]), min(variations[2]) )
    ymax = max( max(variations[0]) , max(variations[1]), max(variations[2]) )
    plt.xlim([0, 24])
    plt.ylim([ymin - 0.001, ymax + 0.001])
    plt.show()

# EPA vs EIA datasets
![alt text](boxplot.png)
![alt text](hist.png)

# ERCOT Generation Visualization
![alt text](TAMU_Texas_generators.png)
![alt text](hour.gif "Renewable Energy Generation by Hour")
![alt text](month.gif "Renewable Energy Generation by Month")

# ERCOT Wind Generation Visualization

In [21]:
widgets.interactive(update_graph, start_day=start_d, start_month=start_m, end_day=end_d, end_month=end_m, location=location)

interactive(children=(Dropdown(description='Start Day', index=14, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, …

# ARIMA Model

In [18]:
widgets.interactive(update_graph1, start_day1=start_d1, location1=location1, length=length1)

interactive(children=(Dropdown(description='Start Day', index=14, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, …

# Automatic Risk Rater

In [20]:
widgets.interactive(update_graph2, start_day2=start_d2)

interactive(children=(Dropdown(description='Start Day', index=26, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, …